In [1]:
# Importing packages and initializing the variable
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.special import erf
import matplotlib.mlab as mlab
import scipy.integrate as integrate
import matplotlib.patches as mpatches
import pylab as P

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data = [ [170, 169, 158, 173, 175, 175, 162, 165, 162, 171, 161, 169, 167, 170],
         [169, 171, 170, 161, 165, 165, 161, 165, 179, 165],
         [160, 159, 171, 166, 169, 161, 172, 162, 169,152,163, 170, 160, 164, 164, 170],
         [150, 160, 162, 159, 162, 155, 160, 159, 155, 159,160, 157, 159, 160, 160],
         [160, 160, 161, 172, 165, 170, 170, 170, 168]
       ] 
Data = np.array(Data)
Data_std = np.zeros_like(Data)
Std = np.zeros(5)
Mean  = np.zeros(5)
T_stat = np.zeros([5,5])
Cov = np.zeros([10,10])

In [2]:
#Calculating the mean and std
for i in range(5):
    Std[i] = np.std(Data[i], ddof=1)
    Mean[i]  = np.average(Data[i])
    Data_std[i]  = (Data[i]-Mean[i])/Std[i]

In [3]:
#Calculating the T-stats
for i in range(5):
    for j in range(5):
        Yi = Mean[i]
        Yj = Mean[j]
        Si = Std[i]
        Sj = Std[j]
        Ni = len(Data[i])
        Nj = len(Data[j])
        T_stat[i][j] = (Yi-Yj)/(math.sqrt((Si**2)/Ni+(Sj**2)/Nj))

In [4]:
#Mapping function - Ignore this
def func(b,a):
    a +=1
    b +=1
    if(a == 1 and b== 2):
        return 0
    elif(a== 1 and b== 3):
        return 1
    elif(a== 1 and b== 4):
        return 2
    elif(a== 1 and b== 5):
        return 3
    elif(a== 2 and b== 3):
        return 4
    elif(a== 2 and b== 4):
        return 5
    elif(a== 2 and b== 5):
        return 6
    elif(a== 3 and b== 4):
        return 7
    elif(a== 3 and b== 5):
        return 8
    elif(a== 4 and b== 5):
        return 9

def rev_func(col):
    for i in range(5):
        for j in range(i):
            if func(i,j) == col:
                return [i,j]

In [5]:
#Using the analytical formulae mentioned in the homework to calculate Covariance Matrix
for i in range(5):
    Si = Std[i]
    Ni = len(Data[i])+0.0
    for j in range(i):
        Sj = Std[j]
        Nj = len(Data[j])+0.0    
        for k in range(5):
            Sk = Std[k]
            Nk = len(Data[k]) +0.0 
            for l in range(k):
                Sl = Std[l]
                Nl = len(Data[l]) +0.0 

                Num = (i==k)*(Si**2/Ni) - (i==l)*(Si**2/Ni) - (j==k)*(Sj**2/Nj) + (j==l)*(Sj**2/Nj)
                Denom = (math.sqrt((Si**2/Ni+Sj**2/Nj)*(Sl**2/Nl+Sk**2/Nk) ))
                Cov[func(i,j)][func(k,l)] = Num/Denom

In [8]:
t_stat = np.zeros(10)
for i in range(5):
    for j in range(i):
        t_stat[func(i,j)] = abs(T_stat[i,j])
t_stat_copy = np.copy(t_stat)
Cov1 = Cov
J = [0,1,2,3,4,5,6,7,8,9]
for k in range(9):
    #Calculate the percentile
    Temp = np.max(np.random.multivariate_normal(np.zeros(10-k), Cov1,100),axis=1)
    limit = np.percentile(Temp,95)
    argmax = np.argmax(t_stat)
    max_val = np.max(t_stat)
    if(max_val > limit ):
        print("Reject", sorted(rev_func(np.where(t_stat_copy==max_val)[0][0])))
    else:
        break
    Cov1 = np.delete(Cov1, (argmax), axis=0)
    Cov1 = np.delete(Cov1, (argmax), axis=1)
    t_stat = np.delete(t_stat, (argmax), axis=0)


('Reject', [0, 3])
('Reject', [1, 3])
('Reject', [3, 4])
('Reject', [2, 3])
